## Matplotlib animations demo using Cepheids

#### This demonstration shows how to make an animated figure with matplotlib. We'll use photometric variability data for a set of Cepheids from Gaia DR1.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

^ The last line there is needed to make the animation render in the notebook.

In [2]:
from get_data import get_cepheids
table = get_cepheids()

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Launched query: 'SELECT  TOP 2000 cep.*, gaia.ra, gaia.dec         FROM gaiadr1.cepheid AS cep         INNER JOIN gaiadr1.gaia_source as gaia         on cep.source_id = gaia.source_id'
Retrieving sync. results...


Query finished.


Based on Cepheid parameters from Gaia DR1, calculate the magnitudes at a given time. Documentation on the parameters can be found [here](https://gea.esac.esa.int/archive/documentation/GDR1/datamodel/Ch2/cepheid.html).

Note that we don't have reference phases supplied, so I'm just making them up!

In [3]:
ref_epochs = table['epoch_g'] + 2455197.5
phis = np.random.rand(len(table)) * 2. * np.pi
A1 = table['peak_to_peak_g']/2.

In [4]:
def delta_mags(epoch):
    delta_t = epoch - ref_epochs
    delta_mags = A1 * np.sin(2. * np.pi * delta_t/table['p1'] + phis) \
        + A1 * table['r21_g'] * np.sin(2. * np.pi * delta_t/table['p1'] 
                                       + phis * table['phi21_g'])
    return delta_mags

Now we make a function to feed to [matplotlib.animation.FuncAnimation](https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html). It takes an epoch and an axis object, clears the axis, and replots the data using the magnitudes at the given epoch as sizes.

In [5]:
mags = table['int_average_g']
sizes = np.exp(mags-15.)

def plot_cepheids(epoch, ax):
    dmags = delta_mags(epoch)
    size_factor = 10.**(dmags/2.5) * 20.
    ax.cla()
    ax.scatter(table['ra'], table['dec'], s=sizes+size_factor)
    ax.set_xlabel('Right Ascension (degrees)', fontsize=16)
    ax.set_ylabel('Declination (degrees)', fontsize=16)
    ax.set_xlim([70,85])
    ax.set_ylim([-68,-62])

Generate some epochs and test it out:

In [6]:
epochs = np.linspace(0, 10, 100) + 2455197.5

In [7]:
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot()
anim = animation.FuncAnimation(fig, plot_cepheids, epochs,
                                fargs=(ax,), 
                                interval=50)
plt.close(fig)
HTML(anim.to_html5_video())

To save this as a gif, we use imagemagick:

In [ ]:
anim.save('cepheids.gif', writer='imagemagick')  

#### Extending this:

A few ideas:
- add a title to the plot that shows the timestamp
- make a full-sky animation for Gaia DR2 (this is more of a data wrangling challenge, as the data model changed)
    - also note that Gaia source_ids change between DRs, so make sure you're getting the correct sky positions!
- put the plot in a better sky projection
- use astropy.coords to apply some exaggerated proper motions/parallaxes to the Cepheids too

#### Further resources:

You can also check out these tutorials:
- [animations of nearby stars using Gaia DR2](https://gist.github.com/eteq/02a0065f15da3b3d8c2a9dea146a2a14) by Erik Tollerud
- [magic triangle animation](https://jakevdp.github.io/blog/2013/05/28/a-simple-animation-the-magic-triangle/) by Jake VanderPlas

And [this poorly-commented notebook](https://github.com/megbedell/gaia-kepler.fun/blob/master/notebooks/demo.ipynb) with the source code for an animation of the Kepler field by yours truly.